## 「[ゼロから作るDeep Learning ❸――フレームワーク編](https://www.oreilly.co.jp/books/9784873119069/)」を元にPythonの基礎解説 第4ステージ
この記事では、前回に続けてPythonの基礎解説と雛形の作成を行なっていきます。DeZeroやNumPyについては詳細な説明は行いません。

### Pythonの参考文献
 - [yield の使い方 - LIFE WITH PYTHON](https://www.lifewithpython.com/2018/05/python-yield.html)
 - [組み込み関数`isinstance` - Python ドキュメント](https://docs.python.org/ja/3/library/functions.html#isinstance)
 - [3. データモデル`object.__setattr__(self, name, value)` - Python ドキュメント](https://docs.python.org/ja/3/reference/datamodel.html#object.__setattr__)
 - [3. データモデル`object.__getitem__(self, key)` - Python ドキュメント](https://docs.python.org/ja/3/reference/datamodel.html#object.__getitem__)
 - [numpy.random — NumPy Manual](https://numpy.org/doc/stable/reference/random/index.html)
   - [numpy.random.rand — NumPy Manual](https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html)：一様分布の乱数生成
   - [numpy.random.randn — NumPy Manual](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html)：正規分布の乱数生成
   - [numpy.random.seed — NumPy Manual](https://numpy.org/doc/stable/reference/random/generated/numpy.random.seed.html)
   - [numpy.random.permutation — NumPy Manual](https://numpy.org/doc/stable/reference/random/generated/numpy.random.permutation.html)
 - [numpy.transpose — NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html), [numpy.ndarray.transpose](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.transpose.html)
 - [numpy.sum — NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.sum.html), [numpy.ndarray.sum](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.sum.html)
 - [numpy.broadcast_to — NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.broadcast_to.html)
 - [numpy.dot — NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.dot.html), [numpy.ndarray.dot](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.dot.html)
 - [numpy.clip — NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.clip.html), [numpy.ndarray.clip](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.clip.html)

## 「第4ステージ ニューラルネットワークを作る」より
「第4ステージ」では、DeZeroをニューラルネットワークに向けて、テンソル（多次元配列）を使って計算できるように拡張され、全結合層・活性化関数の一つのシグモイド関数が提供され、モデルやOptimizerを表すクラスが提供され、クラス分類の確率を表すソフトマックス関数とその損失関数であるクロスエントロピー誤差も提供される。

### 「ステップ37 テンソルを扱う」
### 「ステップ38 形状を変える関数」より
 - NumPyの`np.reshape(x, shape)`, `x.reshape(shape)`で、`ndarray`インスタンス`x`を`shape`の形状へと変換する。`shape`はタプル`(1, 2)`でもリスト`[1, 2]`でも展開した形式`1, 2`でも構わない。
 - 組み込み関数`isinstance(object, classinfo)`の`classinfo`は複数のクラス情報のタプルでも構わない。
   - 参考：[組み込み関数isinstance - Python ドキュメント](https://docs.python.org/ja/3/library/functions.html#isinstance)
 - NumPyの`transpose`関数は多次元配列の0から始まるインデックスで表される軸をインデックスのタプルで指定して入れ替える。インデックスのタプルを指定しなければ軸のインデックスが逆順になる。これは行列の場合は転置行列となる。
   - 参考：[numpy.transpose — NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html)

In [1]:
import numpy as np

x = np.array([[1, 2, 3], [4, 5, 6]])
print(x.reshape((6,))) # np.reshape(x, (6,))と同じ
print(isinstance([1], (tuple, list)))
print(isinstance((1,), (tuple, list)))
print(isinstance(1, (tuple, list)))
x = np.array([[1, 2, 3], [4, 5, 6]])
print(np.transpose(x)) # x.transpose()、x.transpose((1, 0))と同じ
x = np.ones((1, 2, 3))
print(np.transpose(x, (1, 0, 2)).shape) #x.transpose(1, 0, 2).shapeと同じ

[1 2 3 4 5 6]
True
True
False
[[1 4]
 [2 5]
 [3 6]]
(2, 1, 3)


### 「ステップ39 和を求める関数」より
 - NumPyの`np.sum`関数は、引数`axis`に多次元配列の軸のインデックスを指定しなければ（または`axis=None`を指定）和のスカラーを得る。引数`axis`に軸のインデックスを指定すればその軸に和を取り、軸のインデックスのタプルを指定すればその複数の軸に和を取る。引数`keepdims`に`True`を指定すれば、同じ次元数を保つ。
   - 参考：[numpy.sum — NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.sum.html)

In [2]:
import numpy as np

x = np.array([[1, 2, 3], [4, 5, 6]])
print(np.sum(x)) # x.sum()と同じ
print(x.sum(axis=0)) # x.sum(axis=(0,))と同じ
print(x.sum(axis=1)) # x.sum(axis=(1,))と同じ
print(x.sum(keepdims=True)) # shape(1, 1)の行列

21
[5 7 9]
[ 6 15]
[[21]]


### 「ステップ40 ブロードキャストを行う関数」より
 - NumPyの`np.broadcast_to(x, shape)`関数は、`ndarray`インスタンス`x`の要素を`shape`形状に複製する。これをブロードキャストと呼ぶ。ブロードキャストは`ndarray`の演算時に暗黙的に行われることもある。
   - 参考：[numpy.broadcast_to — NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.broadcast_to.html)

In [3]:
import numpy as np

x = np.array([1, 2, 3])
print(np.broadcast_to(x, (2, 3)))

y = np.array([10])
print(x + y)
print(x + np.broadcast_to(y, (3))) #暗黙的ブロードキャスト

[[1 2 3]
 [1 2 3]]
[11 12 13]
[11 12 13]


### 「ステップ41 行列の積」より
 - `ndarray`のベクトルの内積と行列の積は共にNumPyの`np.dot`関数で行う。ベクトルの内積は自動的に内積を行えるようベクトルが転置され、行列の積は行列が自動的に転置されることはなく、積を行える`shape`の行列を`np.dot`関数の引数に渡さなければならない。
   - 参考：[numpy.dot — NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.dot.html)

In [4]:
import numpy as np

A = np.array([[1, 2, 3], [4, 5, 6]])
B = np.array([[1, 2, 3], [4, 5, 6]])
print(np.dot(A, B.transpose()))
print(np.dot(A.transpose(), B))

x = np.array([1, 2, 3])
y = np.array([4, 5, 6])
print(np.dot(x, y))
print(np.dot(x.transpose(), y))

[[14 32]
 [32 77]]
[[17 22 27]
 [22 29 36]
 [27 36 45]]
32
32


### 「ステップ42 線形回帰」より
 - DeZeroの平均2乗誤差クラス・関数`MeanSquaredError`, `mean_squared_error`の実装が解説されている。
 - 機械学習に使用するデータセットで乱数を使う場合は再現性を考慮して、乱数のシードを固定してデータを生成することが多い。

In [5]:
import numpy as np

np.random.seed(0)
print(np.random.rand(5, 1))
print(np.random.rand(5, 1))
np.random.seed(0)
print(np.random.rand(10, 1))

[[0.5488135 ]
 [0.71518937]
 [0.60276338]
 [0.54488318]
 [0.4236548 ]]
[[0.64589411]
 [0.43758721]
 [0.891773  ]
 [0.96366276]
 [0.38344152]]
[[0.5488135 ]
 [0.71518937]
 [0.60276338]
 [0.54488318]
 [0.4236548 ]
 [0.64589411]
 [0.43758721]
 [0.891773  ]
 [0.96366276]
 [0.38344152]]


### 「ステップ43 ニューラルネットワーク」より
 - DeZeroの全結合層に対応する線形変換`linear`関数の実装の解説。
 - 非線形な変換である活性化関数の一つであるシグモイド関数$y=\frac{1}{1+\exp\left(-x\right)}$の実装。
 - シグモイド関数の微分の導出は『ゼロから作るDeep Learning』の「5.5.2 Sigmoidレイヤ」を参照している。
 - 重みの初期値をランダムに設定する必要性の理由は『ゼロから作る Deep Learning』の「6.2.1 重みの初期値を 0 にする？」を参照している。

### 「ステップ44 パラメータをまとめるレイヤ」より
 - DeZeroの`Parameter`クラス：重み・バイアスなどのパラメータを変数Variableと区別するクラス。
 - DeZeroの`Layer`クラス：パラメータを保持し変換するクラス。
 - DeZeroの`Linear`クラス：線形変換を行うレイヤのクラス。ユーザは出力サイズのみ指定し入力サイズは伝搬時に動的に決められる。
 - 特殊メソッド`__setattr___(self, name, value) `はインスタンス変数代入時に、インスタンス変数名`name`、値`value`が渡されて呼び出される。オーバーライドすることでオリジナルの処理を実装できる。
   - 参考：[3. データモデル`object.__setattr__(self, name, value)` - Python ドキュメント](https://docs.python.org/ja/3/reference/datamodel.html#object.__setattr__)
 - インスタンス変数`__dict__`に、すべてのインスタンス変数がディクショナリ形式で格納されている。

In [6]:
class Vehicle:
    def __init__(self, velocity):
        self.velocity = velocity
    
    def run(self):
        raise NotImplementedError()

    def __setattr__(self, name, value):
        super().__setattr__(name, value) # これが無いとプロパティに代入されない
        message = "{} is {}"
        print(message.format(name, value))

class Car(Vehicle):
    @property
    def run(self):
        print('Run at {}.'.format(self.velocity))

c = Car(10) # （__init__で）__setattr__が呼び出される。
c.velocity = 100 # __setattr__が呼び出される。
print(c.__dict__)

velocity is 10
velocity is 100
{'velocity': 100}


### 「ステップ45 レイヤをまとめるレイヤ」より
 - `yield`を使った関数はジェネレータと呼ばれ、返り値はジェネレータイテレータと呼ばれるイテレータであり、呼ばれる度に一つずつ値を返す。ジェネレータから別の新しいジェネレータを作るには、`yield from`を使う。
   - なお、ステップ50でイテレータについて解説される。
   - 参考：[yield の使い方 - LIFE WITH PYTHON](https://www.lifewithpython.com/2018/05/python-yield.html)
 - DeZeroの`Layer`クラスを別の`Layer`も保持する入れ子構造に拡張。`Layer`のパラメータにアクセスする`params`メソッドはジェネレータとして実装されている。
 - DeZeroの`Layer`クラスを継承した`Model`クラスには、計算グラフの可視化メソッド`plot`も実装されており、ニューラルネットワークのモデルを表す。開発者はModelクラスを継承したクラスで個別のネットワークを作成する。
 - DeZeroでは、`Model`クラスを継承した全結合層のニューラルネットワークを表す`MLP`（Multi-Layer Perceptron）クラスが提供されている。

In [7]:
def generator_1():
    for n in 'ABC':
        yield n

def generator_2():
    for n in '123':
        yield n

def generator_all():
    yield from generator_1()
    yield from generator_2()

for i in generator_1():
    print(i)
print(list(generator_2()))
print(list(generator_all()))

A
B
C
['1', '2', '3']
['A', 'B', 'C', '1', '2', '3']


### 「ステップ46 Optimizerによるパラメータ更新」より
 - DeZeroの指定したモデルのパラメータを最適化する基底クラス`Optimizer`。具体的な最適化手法は`Optimizer`クラスを継承して実装する。前処理も設定できる。
 - DeZeroでは、最適化手法SGD（Stochastic Gradient Descent、確率的勾配降下法）、Momentum、AdaGrad、AdaDelta、Adamのクラスを提供。
 - AdaGrad、AdaDelta、Adamなどの最適化手法については『ゼロから作る Deep Learning』の「6.1 パラメータの更新」参照。

### 「ステップ47 ソフトマックス関数と交差エントロピー誤差」より
 - Pythonでは、リストやタプルに対して、`x[1]`や`x[:, 1:4]`のように書くことで多次元配列の一部を抜き出すスライス操作が行える。
 - DeZeroでは、`Variable`の多次元配列から一部を抜き出す関数`get_item`を提供している。
 - Pythonの特殊メソッド`__getitem__`は、オブジェクトに角括弧でアクセスしたときの挙動を定義できる。
 - DeZeroでは、`Variable.__getitem__ = get_item`と設定して、`Variable`オブジェクトに角括弧のスライス操作を与えている。
   - 参考：[3. データモデル`object.__getitem__(self, key)` - Python ドキュメント](https://docs.python.org/ja/3/reference/datamodel.html#object.__getitem__)

In [8]:
class Vehicle:
    def __init__(self, velocity, name):
        self.velocity = velocity
        self.name = name

    def __getitem__(self, key):
        return self.name[key]

veh = Vehicle(10, "qitqito")
print(veh[:3])

qit


 - 分類のクラス数が$n$個の場合、$k\left(1\leq k\leq n\right)$番目の分類確率は、次式のソフトマックス関数で表される。
   - $p_{k}=\frac{\exp\left(y_{k}\right)}{\sum_{i=1}^{n}\exp\left(y_{i}\right)}$、$y_k$：ソフトマックス関数への入力値
   - ソフトマックス関数のより良い実装については、『ゼロから作るDeep Learning』の「3.5.2 ソフトマックス関数の実装上の注意」が参照されている。
 - 分類のクラス数が$n$個の場合、多値分類の損失関数は、交差エントロピー誤差$L=-\sum_{k=1}^{n}t_{k}\log p_{k}$で表される。ここで$p_k$はソフトマックス関数値であり、$t_k$は教師データの`k`次元目の値であり、正解クラスが`1`でその他は`0`となる**one-hot**ベクトルで与えられる。従って、和は実際には1項のみである。
 - 実装では、入力値`x`を`x_min`未満は`x_min`、x_max`を超える場合は`x_max`に置き換える`clip(x, x_min, x_max)`関数が利用されている。
   - 参考：[numpy.clip — NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.clip.html)

In [9]:
import numpy as np

M = np.array([[1, 2, 3], [4, 5, 6]])
print(M[1])
print(M[:, :2])
print(np.clip(0, 1e-15, 1.0))

[4 5 6]
[[1 2]
 [4 5]]
1e-15


### 「ステップ48 多値分類」より
 - データセットのインデックスのシャッフルにはNumPyの`np.random.permutation`関数を使う。
   - 参考：[numpy.random.permutation — NumPy Manual](https://numpy.org/doc/stable/reference/random/generated/numpy.random.permutation.html)

In [10]:
import numpy as np

print(np.random.permutation(10))

[5 2 3 4 1 0 9 8 7 6]


### 「ステップ49 Datasetクラスと前処理」より
 - DeZeroでは前処理の仕組みがあり巨大データセットにも利用できる`Dataset`基底クラスを提供する。`Dataset`クラスは特殊メソッド`__getitem__`、`__len__`（それぞれステップ47、19で解説）を実装している。


### 「ステップ50 ミニバッチを取り出すDataLoader」より
 - DeZeroの`Dataset`クラスからミニバッチ作成やデータセットのシャッフルを行う`Dataloader`クラスを実装。
 - Pythonのイテレータは、リストやタプルなど複数の要素を持つデータ型に対して、順番にデータを取り出す機能を提供する。`iter`関数によりリストをイテレータに変換する。`next`関数によりイテレータに対して順にデータを取り出す。次の要素が存在しない場合は`StopIteration`例外が発生する。

In [11]:
t = [1, 2, 3]
x = iter(t) # listからイテレータ生成
print(next(x))
print(next(x))
print(next(x))
print(next(x)) # StopIteration例外発生

1
2
3


StopIteration: 

In [12]:
t = [1, 2, 3]
for x in t: # リストtはイテレータに変換される。StopIteration発生でfor文終了。
    print(x)

1
2
3


 - Pythonのイテレータは、自分自身（`self`）を返す`__iter__`特殊メソッド、次の要素を返す`__next__`特殊メソッド（返す要素がなければ`raise StopIteration()`）を実装することで自作できる。

In [13]:
class Vehicle:
    def __init__(self, velocity, name):
        self.velocity = velocity
        self.name = name

    def __repr__(self):
        return 'Velocity is {}.'.format(self.velocity)

    def __iter__(self):
        return self

    def __next__(self):
        if self.velocity >= 110:
            raise StopIteration()
        self.velocity += 1
        return self.velocity

class Car(Vehicle):
    @property
    def run(self):
        print('Run at {}.'.format(self.velocity))

c = Car(100, "TES")
for i in c:
    print(i)

101
102
103
104
105
106
107
108
109
110


 - `numpy.argmax`, `numpy.ndarray.argmax`は、最大値のインデックスを返す。`axis`を指定すれば、軸に沿った最大値のインデックスを返す。
   - 参考：[numpy.argmax — NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html), [numpy.ndarray.argmax — NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.argmax.html)
 - `numpy.ndarray`配列を条件式にすれば、要素毎の真偽により`True, False`の配列が得られる。
 - `numpy.mean`で平均値を求める。
   - 参考：[numpy.mean — NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)

In [14]:
import numpy as np

ar = np.array(range(12)).reshape(3,4)
print(ar)
ar_6 = (ar >= 6)
print(ar_6)
print(ar_6.mean())
print(ar.argmax())
print(ar.argmax(axis=0))
print(ar.argmax(axis=1))

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[False False False False]
 [False False  True  True]
 [ True  True  True  True]]
0.5
11
[2 2 2 2]
[3 3 3]


### 「ステップ51 MNISTの学習」より
 - DeZeroによるMNISTのソースが掲載されており、全結合層２層で86%強の精度、全結合層３層で活性化関数をReLUにして最適化手法をSGDからAdamに変えると98%の精度が得られる。
   - 参考：[deep-learning-from-scratch-3/examples/mnist.py](https://github.com/oreilly-japan/deep-learning-from-scratch-3/blob/master/examples/mnist.py)
   - 参考：[pytorch/examples/mnist/main.py](https://github.com/pytorch/examples/blob/master/mnist/main.py)